In [ ]:
import cv2 as cv
from cv2 import VideoCapture
import numpy as np
from typing import Callable, Tuple

In [ ]:
cap = VideoCapture()


class ViewController:
    @staticmethod
    def midpoint_to_corner(mid_y: int, mid_x: int, h: int, w: int) -> Tuple[int, int, int, int]:
        cor_y = mid_y - h // 2
        cor_x = mid_x - w // 2
        return cor_y, cor_x, h, w

    @staticmethod
    def corner_to_midpoint(cor_y: int, cor_x: int, h: int, w: int) -> Tuple[int, int, int, int]:
        mid_y = cor_y + h // 2
        mid_x = cor_x + w // 2
        return mid_y, mid_x, h, w

    def __init__(
        self,
        video_stream: VideoCapture,
        camera_size: Tuple[int, int] = (201, 201),
        micro_size: Tuple[int, int] = (41, 41),
        init_position: Tuple[int, int] = (0, 0),
        padding_value=[255, 255, 255],
    ) -> None:
        self._video_stream = video_stream
        self._camera_size = camera_size
        self._micro_size = micro_size

        cam_height, cam_width = camera_size
        self._padding_size = [cam_height // 2, cam_width // 2]
        self._padding_value = padding_value

        frame_width = int(video_stream.get(cv.CAP_PROP_FRAME_WIDTH))
        frame_height = int(video_stream.get(cv.CAP_PROP_FRAME_HEIGHT))
        self._frame_size = (frame_height, frame_width)

        self.set_position(*init_position)

        self._frame = None
        res = self.next_frame()
        if res is False:
            raise Exception("couldn't read first frame")

    def skip_frames(self, count: int) -> bool:
        for _ in range(count):
            res, _ = self._video_stream.read()
            if res is False:
                break
        return self.next_frame()

    def next_frame(self) -> bool:
        ret, frame = self._video_stream.read()
        if ret == False:
            self._frame = None
            return False

        self._frame = cv.copyMakeBorder(
            src=frame,
            top=self._padding_size[0],
            bottom=self._padding_size[0],
            left=self._padding_size[1],
            right=self._padding_size[1],
            borderType=cv.BORDER_CONSTANT,
            value=self._padding_value,
        )

        return True

    def frame_size(self) -> Tuple[int, int]:
        return self._frame_size

    def camera_size(self) -> Tuple[int, int]:
        return self._camera_size

    def micro_size(self) -> Tuple[int, int]:
        return self._micro_size

    def position(self) -> Tuple[int, int]:
        return self._position

    def set_position(self, y: int, x: int):
        assert y >= 0 and y < self._frame_size[0]
        assert x >= 0 and x < self._frame_size[1]
        self._position = (y, x)

    def move_position(self, dy: int, dx: int):
        self.set_position(self._position[0] + dy, self._position[1] + dx)

    def camera_view(self) -> np.ndarray:
        y, x, h, w = ViewController.midpoint_to_corner(*self._position, *self.camera_size())

        # Take padding into account
        h_pad, w_pad = self._padding_size
        y, x = y + h_pad, x + w_pad

        slice = self._frame[y : y + h, x : x + w]
        return slice

    def micro_view(self) -> np.ndarray:
        y, x, h, w = ViewController.midpoint_to_corner(*self._position, *self.micro_size())

        # Take padding into account
        h_pad, w_pad = self._padding_size
        y, x = y + h_pad, x + w_pad

        slice = self._frame[y : y + h, x : x + w]
        return slice

    def visualize_world(self):
        h_pad, w_pad = self._padding_size

        y_cam, x_cam, h_cam, w_cam = ViewController.midpoint_to_corner(*self._position, *self.camera_size())
        y_cam, x_cam = y_cam + h_pad, x_cam + w_pad

        y_mic, x_mic, h_mic, w_mic = ViewController.midpoint_to_corner(*self._position, *self.micro_size())
        y_mic, x_mic = y_mic + h_pad, x_mic + w_pad

        image = self._frame.copy()
        cv.rectangle(image, (x_cam, y_cam), (x_cam + w_cam, y_cam + h_cam), (0, 0, 255), 5)
        cv.rectangle(image, (x_mic, y_mic), (x_mic + w_mic, y_mic + h_mic), (0, 255, 0), 5)

        cv.imshow("world view", image)

    def close_windows(self):
        cv.destroyAllWindows()

In [ ]:
cap = VideoCapture("worms.avi")

sim = ViewController(cap)

while sim.next_frame():
    view = sim.micro_view()
    cv.imshow("view", view)
    #sim.visualize_world()
    #sim.move_position(5, 5)


    if cv.waitKey(1) & 0xFF == ord("q"):
        break

sim.close_windows()

In [ ]:
cv.destroyAllWindows()